# NebulaGraphQAChain

这个笔记本展示了如何使用LLM为NebulaGraph数据库提供自然语言接口。

你需要一个运行中的NebulaGraph集群，可以通过运行以下脚本来运行容器化的集群：
```bash
curl -fsSL nebula-up.siwei.io/install.sh | bash
```

其他选项包括：- 安装为[Docker Desktop Extension]（https://www.docker.com/blog/distributed-cloud-native-graph-database-nebulagraph-docker-extension/）。请参见[此处](https://docs.nebula-graph.io/3.5.0/2.quick-start/1.quick-start-workflow/)- NebulaGraph云服务。请参见[此处]（https://www.nebula-graph.io/cloud）- 通过软件包、源代码或通过Kubernetes部署。请参见[此处](https://docs.nebula-graph.io/)
一旦集群运行，我们就可以为数据库创建SPACE和SCHEMA。

In [ ]:
%pip install ipython-ngql
%load_ext ngql

# connect ngql jupyter extension to nebulagraph
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula
# create a new space
%ngql CREATE SPACE IF NOT EXISTS langchain(partition_num=1, replica_factor=1, vid_type=fixed_string(128));


In [ ]:
# Wait for a few seconds for the space to be created.
%ngql USE langchain;

创建模式，请参阅[此处]（https://www.siwei.io/en/nebulagraph-etl-dbt/）以获取完整数据集。

In [ ]:
%%ngql
CREATE TAG IF NOT EXISTS movie(name string);
CREATE TAG IF NOT EXISTS person(name string, birthdate string);
CREATE EDGE IF NOT EXISTS acted_in();
CREATE TAG INDEX IF NOT EXISTS person_index ON person(name(128));
CREATE TAG INDEX IF NOT EXISTS movie_index ON movie(name(128));

等待模式创建完成，然后我们可以插入一些数据。

In [1]:
%%ngql
INSERT VERTEX person(name, birthdate) VALUES "Al Pacino":("Al Pacino", "1940-04-25");
INSERT VERTEX movie(name) VALUES "The Godfather II":("The Godfather II");
INSERT VERTEX movie(name) VALUES "The Godfather Coda: The Death of Michael Corleone":("The Godfather Coda: The Death of Michael Corleone");
INSERT EDGE acted_in() VALUES "Al Pacino"->"The Godfather II":();
INSERT EDGE acted_in() VALUES "Al Pacino"->"The Godfather Coda: The Death of Michael Corleone":();

UsageError: Cell magic `%%ngql` not found.


In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import NebulaGraphQAChain
from langchain.graphs import NebulaGraph

In [2]:
graph = NebulaGraph(
    space="langchain",
    username="root",
    password="nebula",
    address="127.0.0.1",
    port=9669,
    session_pool_size=30,
)

## 刷新图形模式信息

如果数据库的模式发生更改，您可以刷新所需的模式信息以生成nGQL语句。

In [ ]:
# graph.refresh_schema()

In [3]:
print(graph.get_schema)

Node properties: [{'tag': 'movie', 'properties': [('name', 'string')]}, {'tag': 'person', 'properties': [('name', 'string'), ('birthdate', 'string')]}]
Edge properties: [{'edge': 'acted_in', 'properties': []}]
Relationships: ['(:person)-[:acted_in]->(:movie)']



## 查询图形

现在我们可以使用图形cypher QA链来询问图形。

In [5]:
chain = NebulaGraphQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)


In [6]:
chain.run("Who played in The Godfather II?")



> Entering new NebulaGraphQAChain chain...
Generated nGQL:
MATCH (p:`person`)-[:acted_in]->(m:`movie`) WHERE m.`movie`.`name` == 'The Godfather II'
RETURN p.`person`.`name`
Full Context:
{'p.person.name': ['Al Pacino']}

> Finished chain.


'Al Pacino played in The Godfather II.'